# Chapter 15: Creational and Structural Patterns

This notebook covers the classic creational and structural design patterns and their Pythonic
implementations. Python's dynamic nature and first-class functions often simplify these patterns
compared to their traditional Java/C++ forms.

## Key Concepts
- **Factory pattern**: Registry-based object creation
- **Builder pattern**: Step-by-step object construction
- **Singleton pattern**: Single instance (and why modules are often better)
- **Adapter pattern**: Making incompatible interfaces work together
- **Decorator pattern**: Adding behavior without modifying objects (OOP, not Python decorators)
- **Facade pattern**: Simplified interface to complex subsystems
- **Pythonic alternatives**: Functions, modules, and `__init__.py`

## Factory Pattern

The Factory pattern encapsulates object creation logic. Instead of calling constructors directly,
a factory function or class decides which type to instantiate. In Python, a **registry dict**
mapping names to classes is a clean and extensible approach.

In [ ]:
from abc import ABC, abstractmethod


# Base class using ABC
class Shape(ABC):
    @abstractmethod
    def area(self) -> float: ...

    @abstractmethod
    def describe(self) -> str: ...


class Circle(Shape):
    def __init__(self, radius: float) -> None:
        self.radius = radius

    def area(self) -> float:
        return 3.14159 * self.radius ** 2

    def describe(self) -> str:
        return f"Circle(radius={self.radius})"


class Square(Shape):
    def __init__(self, side: float) -> None:
        self.side = side

    def area(self) -> float:
        return self.side ** 2

    def describe(self) -> str:
        return f"Square(side={self.side})"


class Triangle(Shape):
    def __init__(self, base: float, height: float) -> None:
        self.base = base
        self.height = height

    def area(self) -> float:
        return 0.5 * self.base * self.height

    def describe(self) -> str:
        return f"Triangle(base={self.base}, height={self.height})"


# Registry-based factory
shape_registry: dict[str, type[Shape]] = {
    "circle": Circle,
    "square": Square,
    "triangle": Triangle,
}


def create_shape(kind: str, **kwargs) -> Shape:
    """Factory function that creates shapes by name."""
    if kind not in shape_registry:
        raise ValueError(f"Unknown shape: {kind!r}. Available: {list(shape_registry.keys())}")
    return shape_registry[kind](**kwargs)


# Create shapes without knowing the concrete classes
shapes = [
    create_shape("circle", radius=5.0),
    create_shape("square", side=4.0),
    create_shape("triangle", base=6.0, height=3.0),
]

for shape in shapes:
    print(f"{shape.describe()} -> area = {shape.area():.2f}")

# The registry is extensible - add new shapes without modifying create_shape
print(f"\nRegistered shapes: {list(shape_registry.keys())}")
print(f"isinstance check: {isinstance(shapes[0], Circle)}")
print(f"Square area: {shapes[1].area()}")

## Builder Pattern

The Builder pattern constructs complex objects step by step. It is especially useful when an
object has many optional parameters. In Python, builders often use **method chaining** (returning
`self` from each method).

In [ ]:
from dataclasses import dataclass, field


@dataclass
class HttpRequest:
    """Complex object with many optional fields."""
    method: str = "GET"
    url: str = ""
    headers: dict[str, str] = field(default_factory=dict)
    body: str | None = None
    timeout: int = 30
    retries: int = 0


class HttpRequestBuilder:
    """Step-by-step construction with method chaining."""

    def __init__(self) -> None:
        self._method = "GET"
        self._url = ""
        self._headers: dict[str, str] = {}
        self._body: str | None = None
        self._timeout = 30
        self._retries = 0

    def method(self, method: str) -> "HttpRequestBuilder":
        self._method = method
        return self

    def url(self, url: str) -> "HttpRequestBuilder":
        self._url = url
        return self

    def header(self, key: str, value: str) -> "HttpRequestBuilder":
        self._headers[key] = value
        return self

    def body(self, body: str) -> "HttpRequestBuilder":
        self._body = body
        return self

    def timeout(self, seconds: int) -> "HttpRequestBuilder":
        self._timeout = seconds
        return self

    def retries(self, count: int) -> "HttpRequestBuilder":
        self._retries = count
        return self

    def build(self) -> HttpRequest:
        if not self._url:
            raise ValueError("URL is required")
        return HttpRequest(
            method=self._method,
            url=self._url,
            headers=self._headers,
            body=self._body,
            timeout=self._timeout,
            retries=self._retries,
        )


# Fluent, readable construction
request = (
    HttpRequestBuilder()
    .method("POST")
    .url("https://api.example.com/users")
    .header("Content-Type", "application/json")
    .header("Authorization", "Bearer token123")
    .body('{"name": "Alice"}')
    .timeout(10)
    .retries(3)
    .build()
)

print(f"Method:  {request.method}")
print(f"URL:     {request.url}")
print(f"Headers: {request.headers}")
print(f"Body:    {request.body}")
print(f"Timeout: {request.timeout}s")
print(f"Retries: {request.retries}")

## Singleton Pattern (and Why Modules Are Often Better)

The Singleton pattern ensures only one instance of a class exists. In Python, **modules are
natural singletons** - they are loaded once and cached in `sys.modules`. For most use cases,
a module-level variable is simpler and more Pythonic than a Singleton class.

In [ ]:
# Approach 1: Classic Singleton using __new__
class SingletonMeta(type):
    """Metaclass that ensures only one instance exists."""
    _instances: dict[type, object] = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]


class DatabaseConnection(metaclass=SingletonMeta):
    def __init__(self, host: str = "localhost") -> None:
        self.host = host
        self.connected = True

    def query(self, sql: str) -> str:
        return f"Executing on {self.host}: {sql}"


# Both variables point to the same instance
db1 = DatabaseConnection("prod-server")
db2 = DatabaseConnection("other-server")  # Ignored - already created

print(f"db1 is db2: {db1 is db2}")
print(f"db1.host: {db1.host}")
print(f"db2.host: {db2.host}")
print(f"Same id: {id(db1) == id(db2)}")


# Approach 2: Pythonic alternative - just use a module variable
# In a real project, you would put this in a module like config.py:
#
# config.py
# settings = {"debug": False, "db_host": "localhost", "db_port": 5432}
#
# Then import it anywhere:
# from config import settings
#
# The module is loaded once and cached - natural singleton!
print("\nPythonic approach: use module-level variables instead of Singleton classes")
print("  - Modules are cached in sys.modules after first import")
print("  - Simpler, no metaclass magic needed")
print("  - Easier to test (just patch the module variable)")

## Adapter Pattern

The Adapter pattern makes incompatible interfaces work together. It wraps an existing class
with a new interface that clients expect. This is common when integrating third-party libraries.

In [ ]:
from typing import Protocol


# The interface our application expects
class PaymentProcessor(Protocol):
    def charge(self, amount: float, currency: str) -> str: ...


# Third-party library with a different interface (we cannot modify it)
class LegacyPaymentGateway:
    """Old API with a completely different method signature."""

    def make_payment(self, cents: int, curr_code: str) -> dict:
        return {
            "status": "success",
            "charged": cents,
            "currency": curr_code,
        }


class NewPaymentApi:
    """Modern API - also has a different interface."""

    def process(self, amount_str: str) -> str:
        return f"Processed payment of {amount_str}"


# Adapters translate between interfaces
class LegacyPaymentAdapter:
    """Adapts LegacyPaymentGateway to PaymentProcessor interface."""

    def __init__(self, gateway: LegacyPaymentGateway) -> None:
        self._gateway = gateway

    def charge(self, amount: float, currency: str) -> str:
        cents = int(amount * 100)
        result = self._gateway.make_payment(cents, currency)
        return f"{result['status']}: {result['charged']} {result['currency']}"


class NewPaymentAdapter:
    """Adapts NewPaymentApi to PaymentProcessor interface."""

    def __init__(self, api: NewPaymentApi) -> None:
        self._api = api

    def charge(self, amount: float, currency: str) -> str:
        return self._api.process(f"{currency} {amount:.2f}")


def process_order(processor: PaymentProcessor, total: float) -> None:
    """Works with any PaymentProcessor - doesn't know about adapters."""
    result = processor.charge(total, "USD")
    print(f"  Payment result: {result}")


print("Using legacy gateway via adapter:")
legacy = LegacyPaymentAdapter(LegacyPaymentGateway())
process_order(legacy, 29.99)

print("\nUsing new API via adapter:")
modern = NewPaymentAdapter(NewPaymentApi())
process_order(modern, 29.99)

## Decorator Pattern (OOP Pattern, Not Python Decorators)

The Decorator pattern adds behavior to objects dynamically by wrapping them. This is the
**Gang of Four** Decorator - not to be confused with Python's `@decorator` syntax (though
they share the concept of wrapping).

In [ ]:
from abc import ABC, abstractmethod


class Notifier(ABC):
    @abstractmethod
    def send(self, message: str) -> list[str]: ...


class BasicNotifier(Notifier):
    """Base notifier - sends via app notification."""

    def send(self, message: str) -> list[str]:
        return [f"App: {message}"]


class NotifierDecorator(Notifier):
    """Base decorator - wraps another notifier."""

    def __init__(self, wrapped: Notifier) -> None:
        self._wrapped = wrapped

    def send(self, message: str) -> list[str]:
        return self._wrapped.send(message)


class EmailDecorator(NotifierDecorator):
    """Adds email notification on top of existing behavior."""

    def send(self, message: str) -> list[str]:
        results = super().send(message)
        results.append(f"Email: {message}")
        return results


class SmsDecorator(NotifierDecorator):
    """Adds SMS notification on top of existing behavior."""

    def send(self, message: str) -> list[str]:
        results = super().send(message)
        results.append(f"SMS: {message}")
        return results


class SlackDecorator(NotifierDecorator):
    """Adds Slack notification on top of existing behavior."""

    def send(self, message: str) -> list[str]:
        results = super().send(message)
        results.append(f"Slack: {message}")
        return results


# Stack decorators to combine notification channels
notifier = BasicNotifier()
print("Basic only:")
for msg in notifier.send("Server is down!"):
    print(f"  {msg}")

# Wrap with email and SMS
notifier = SmsDecorator(EmailDecorator(BasicNotifier()))
print("\nWith Email + SMS:")
for msg in notifier.send("Server is down!"):
    print(f"  {msg}")

# Wrap with all channels
notifier = SlackDecorator(SmsDecorator(EmailDecorator(BasicNotifier())))
print("\nWith Email + SMS + Slack:")
for msg in notifier.send("Server is down!"):
    print(f"  {msg}")

## Facade Pattern

The Facade pattern provides a **simplified interface** to a complex subsystem. Clients interact
with one easy-to-use class instead of many interconnected classes.

In [ ]:
# Complex subsystem with multiple classes
class InventorySystem:
    def __init__(self) -> None:
        self._stock: dict[str, int] = {"widget": 100, "gadget": 50}

    def check_stock(self, item: str) -> int:
        return self._stock.get(item, 0)

    def reserve(self, item: str, qty: int) -> bool:
        if self._stock.get(item, 0) >= qty:
            self._stock[item] -= qty
            return True
        return False


class PaymentSystem:
    def process_payment(self, amount: float) -> dict:
        return {"status": "charged", "amount": amount}


class ShippingSystem:
    def create_shipment(self, item: str, qty: int, address: str) -> str:
        return f"Shipping {qty}x {item} to {address}"


class NotificationSystem:
    def send_confirmation(self, email: str, details: str) -> str:
        return f"Email to {email}: {details}"


# Facade simplifies the interaction
class OrderFacade:
    """One simple interface to the entire order subsystem."""

    def __init__(self) -> None:
        self._inventory = InventorySystem()
        self._payment = PaymentSystem()
        self._shipping = ShippingSystem()
        self._notifications = NotificationSystem()

    def place_order(
        self, item: str, qty: int, price: float, address: str, email: str
    ) -> dict:
        """Single method replaces complex multi-step workflow."""
        # Step 1: Check and reserve inventory
        stock = self._inventory.check_stock(item)
        if stock < qty:
            return {"success": False, "error": f"Only {stock} in stock"}

        self._inventory.reserve(item, qty)

        # Step 2: Process payment
        payment = self._payment.process_payment(price * qty)

        # Step 3: Create shipment
        shipment = self._shipping.create_shipment(item, qty, address)

        # Step 4: Send confirmation
        confirmation = self._notifications.send_confirmation(email, shipment)

        return {
            "success": True,
            "payment": payment,
            "shipment": shipment,
            "confirmation": confirmation,
        }


# Client code is simple - one call does everything
orders = OrderFacade()
result = orders.place_order("widget", 3, 9.99, "123 Main St", "alice@example.com")

print("Order result:")
for key, value in result.items():
    print(f"  {key}: {value}")

## Pythonic Alternatives: Functions, Modules, and `__init__.py`

Many GoF patterns exist to work around limitations of languages like Java (no first-class
functions, no modules as objects). Python's features often provide simpler alternatives.

In [ ]:
from typing import Callable


# Instead of Factory class, use a simple function or dict
def make_greeter(style: str) -> Callable[[str], str]:
    """Factory that returns functions - no class hierarchy needed."""
    greeters: dict[str, Callable[[str], str]] = {
        "formal": lambda name: f"Good day, {name}. How do you do?",
        "casual": lambda name: f"Hey {name}!",
        "silent": lambda name: f"...",
    }
    return greeters.get(style, greeters["casual"])


greet = make_greeter("formal")
print(f"Formal: {greet('Alice')}")

greet = make_greeter("casual")
print(f"Casual: {greet('Bob')}")


# Instead of Builder class, use keyword arguments + dataclass
from dataclasses import dataclass


@dataclass
class Config:
    """Pythonic builder alternative: dataclass with defaults."""
    host: str = "localhost"
    port: int = 8080
    debug: bool = False
    workers: int = 4
    log_level: str = "INFO"


# No builder needed - keyword arguments are clear and flexible
dev_config = Config(debug=True, log_level="DEBUG", workers=1)
prod_config = Config(host="0.0.0.0", workers=16)

print(f"\nDev:  {dev_config}")
print(f"Prod: {prod_config}")


# Instead of Singleton, use module-level state
# (Imagine this is in its own file: app_state.py)
class _AppState:
    def __init__(self) -> None:
        self.initialized = False
        self.settings: dict[str, str] = {}

    def init(self, **settings: str) -> None:
        self.settings = settings
        self.initialized = True


# Module-level instance - natural singleton
app_state = _AppState()
app_state.init(env="production", region="us-east-1")
print(f"\nApp state: {app_state.settings}")
print(f"Initialized: {app_state.initialized}")

# __init__.py as Facade:
# In a package's __init__.py, you can re-export selected names
# to present a clean public API:
#
# mypackage/__init__.py:
#   from mypackage.core import Engine
#   from mypackage.config import Settings
#   __all__ = ["Engine", "Settings"]
#
# Users just do: from mypackage import Engine
print("\n__init__.py acts as a Facade for packages")
print("  - Re-export the public API from submodules")
print("  - Hide internal complexity behind clean imports")

## Summary

### Creational Patterns
- **Factory**: Use a registry dict mapping names to classes; call `registry[key](**kwargs)`
- **Builder**: Use method chaining for complex construction; or just use `@dataclass` with defaults
- **Singleton**: Use module-level variables instead of metaclass tricks

### Structural Patterns
- **Adapter**: Wrap incompatible third-party interfaces to match your Protocol
- **Decorator (OOP)**: Stack wrappers to compose behavior; each layer delegates to the wrapped object
- **Facade**: Hide complex subsystems behind one simple class

### Pythonic Alternatives
- First-class functions replace many class-based patterns (Factory, Strategy)
- `@dataclass` with keyword arguments often replaces Builder
- Module-level state replaces Singleton
- `__init__.py` re-exports serve as a natural Facade for packages
- When in doubt, start simple and add patterns only when complexity demands it